<h3>Data Cleansing en el archivo .CSV "Building_Permits.csv"</h3><br>

<b>Ejercicio</b>: Valores faltantes (missing values)<br>


<hr><br>
Importamos las librerias con las que trabajar(ya instaladas).

In [97]:
import pandas as pd
import numpy as np

Leemos el archivo que vamos a analizar. Se encuentra en esta misma carpeta.

In [98]:
building_df = pd.read_csv('Building_Permits.csv', low_memory = False)

<hr> Lo primero que debemos hacer con un dataset nuevo es observar una muestra de sus datos. Esto nos permite ver que todo se leyó correctamente y nos da una idea de lo que sucede en estos. En nuestro caso, vamos a ver si faltan valores, que se representarán con NaN o None.<br><br>
 · Para esto usamos el metodo <b>.head()</b>, que nos deja ver los primeros cinco datos.<br> ·También podriamos usar el metodo <b>.sample()</b>, que muestra un dato aleatorio dentro del conjunto de datos, con el parametro "5" o el numero que queramos de muestras.

In [99]:
building_df.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,NaN,Geary,St,...,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,NaN,Pacific,Av,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,NaN,Pacific,Av,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,NaN,Market,St,...,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992


<br>
Vemos que efectivamente hay datos NaN o valor nulo en la columna 'Street Number Suffix'.
<br>

<b> ¿Cúantos valores nulos hay en nuestro DataFrame?</b>

· Creamos una variable con la suma total de todos los valores nulos, usando los metodos <b>.isnull()</b> y <b>.sum()</b>, que resulta en un objeto Serie.

In [100]:
total_valores_null = building_df.isnull().sum()
total_valores_null.sort_values(ascending = False, inplace = True) #Codigo para ordenar la serie

In [101]:
total_valores_null[0:10] #Vemos las 10 columnas con más valores NaN

TIDF Compliance                  198898
Voluntary Soft-Story Retrofit    198865
Unit Suffix                      196939
Street Number Suffix             196684
Site Permit                      193541
Structural Notification          191978
Fire Only Permit                 180073
Unit                             169421
Completed Date                   101709
Permit Expiration Date            51880
dtype: int64

<hr>
Usando <b>.shape</b>, podemos ver que el DataFrame se compone de 198900 filas y 43 columnas, lo que significa que casí el 100% de la columna TIDF Compliance esta rellenada con valores NaN.

· Podemos ver el porcentaje especifico de valores faltantes en el DataFrame

In [102]:
total_celdas = np.product(building_df.shape)
total_datos_faltantes = total_valores_null.sum()
porcentaje_de_nulos = (total_datos_faltantes/total_celdas) * 100

print("El porcentaje de valores nulos es de:", round(porcentaje_de_nulos, 5),  "%")

El porcentaje de valores nulos es de: 26.26002 %


<br>Vemos que <b>más de una cuarta parte</b> de los datos en el DataFrame corresponden a valores nulos.

<h2>Averiguar por qué faltan los datos</h2>
<p>Ahora que sabemos que un gran porcentaje de estos datos son nulos, se debe de averiguar el <i>por qué</i> es de esta forma.</p>
 En este momento hay que preguntarse:
    <ul><li>¿El valor falta porque no se ha registrado o porque no existe?</li>
</ul>
<p>Por ejemplo, si falta un valor porque no existia(o existe) en el momento en el que se registraron los datos, no tiene sentido intentar adivinar o inferir <i>cúal podría ser</i></p>

En cambio, si faltase un valor que podia ser registrado y no se registro, se puede intentar adivinar <i>cúal podría haber sido</i> en función de los otros valores en esa columna y fila, y esto se conoce por el nombre <b>imputación</b>.<br> <br>
En estadistica, la <b>imputación</b> es la sustitución de valores que no han sido informados en una observación por otros valores.



        building_df['Completed Date'].isnull().sum()
    --> 101709

 Por ejemplo, la columna 'Completed Date' contiene 101.709 valores nulos. La intuición nos dice que corresponden a valores que no se pudieron registrar debido a que todavía no se ha completado la construcción de estas propiedades.

In [103]:
columna_muchos_null=[]
for i in total_valores_null[0:8].index:
    columna_muchos_null.append(i)
    print(i)

TIDF Compliance
Voluntary Soft-Story Retrofit
Unit Suffix
Street Number Suffix
Site Permit
Structural Notification
Fire Only Permit
Unit


In [104]:
building_df.drop(columna_muchos_null, axis = 1, inplace = True) #Quito columnas con poca utilidad y con muchos datos faltantes.
#uso el parametro inplace en True para cambiar el DataFrame 

In [ ]:
total_valores_null = building_df.isnull().sum() #Vuelvo a contar los nulls ahora que he borrado las columnas con más cantidad
total_valores_null

Permit Number                                  0
Permit Type                                    0
Permit Type Definition                         0
Permit Creation Date                           0
Block                                          0
Lot                                            0
Street Number                                  0
Street Name                                    0
Street Suffix                               2768
Description                                  290
Current Status                                 0
Current Status Date                            0
Filed Date                                     0
Issued Date                                14940
Completed Date                            101709
First Construction Document Date           14946
Number of Existing Stories                 42784
Number of Proposed Stories                 42868
Permit Expiration Date                     51880
Estimated Cost                             38066
Revised Cost        

<hr>
<h2>Quitar los valores faltantes(o NaN): .dropna</h2>
<p>Otra forma de encargarse de valores faltantes o del tipo NaN es utilizar la función <b>.dropna</b> para remover columnas o filas. Esta no es una practica recomendada a seguir, especialmente en proyectos importantes. Usualmente vale más el trabajo de explorar nuestros datos, y ver cada una de las columnas con valores ausentes para conocer nuestro mejor el conjunto de datos(<i>dataset</i>).</p> 
<br>
 ·Sin embargo, si estamos totalmente seguros a realizar esta acción, pandas nos ofrece esta función que remueve todas las filas con un valor ausente.


In [121]:
building_df.dropna().shape

(86817, 35)

Vemos que nuestro DataFrame ahora tiene cada una de las filas con datos completos: en total se cuentan <b>86817 filas</b>. Lo que significa que, haciendo el siguiente calculo:

In [123]:
building_df.shape[0]-building_df.dropna().shape[0]

112083

Podemos ver que la función .dropna() nos quita 112083 filas con valores NaN. Esto podría haber correspondido con casi el 100% las filas del dataframe si no hubieramos realizado el paso de quitar las columnas 'TIDF Compliance' o 'Voluntary Soft-Story Retrofit'.<br>
Para que la función .dropna() reemplace los valores del dataframe al que indica la variable, tenemos que usar el parametro 'inplace = True'.<br> De lo contrario, ejecutar la función no tiene efecto en el DF. <br>
<br>
Podemos también remover por columna que tenga valores NaN, para eso usamos el parametro <b> axis</b> en valor 1.<br>
En este caso vemos que quedan las 198900 filas totales, y 12 columnas sin valores faltantes.

In [150]:
building_df_sin_col_n = building_df.dropna(axis=1)
building_df_sin_col_n.head() #función head para ver los 5 primeros resultados

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Current Status,Current Status Date,Filed Date,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,Ellis,expired,12/21/2017,05/06/2015,1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,Geary,issued,08/03/2017,04/19/2016,1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,Pacific,withdrawn,09/26/2017,05/27/2016,1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,Pacific,complete,07/24/2017,11/07/2016,1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,Market,issued,12/01/2017,11/28/2016,144548169992


In [151]:
print('Forma resultante:', building_df_sin_col_n.shape) #total de 198900 filas y 12 columnas
print('Columnas eliminadas:', building_df.shape[1]-building_df_sin_col_n.shape[1]) #Vemos que la función dropna() nos ha quitado 23 columnas
print('Celdas totales eliminadas:', (building_df.shape[1]-building_df_sin_col_n.shape[1]) * building_df_sin_col_n.shape[0])
print('Celdas que nos quedaron:', (building_df_sin_col_n.shape[0] * building_df_sin_col_n.shape[1]))

Forma resultante: (198900, 12)
Columnas eliminadas: 23
Celdas totales eliminadas: 4574700
Celdas que nos quedaron: 2386800


<hr>
<h3>Llenar valores faltantes con la función .fillna()</h3>

Otra opción que tenemos para tratar los datos faltantes es usar la función .fillna() para que Pandas reemplace automaticamente cada valor NaN en el DataFrame. Por ejemplo, si queremos reemplazar con el valor cero:

In [207]:
columnas = ['Permit Number', 'Permit Type', 'Permit Creation Date', 'Description', 'Revised Cost']
building_df_2 = building_df[columnas] #Creo otro dataFrame a partir del dataFrame original.
building_df_2.isna().sum() #contamos cúantos valores nulos hay

Permit Number              0
Permit Type                0
Permit Creation Date       0
Description              290
Revised Cost            6066
dtype: int64

In [208]:
building_df_2[building_df_2['Revised Cost'].isna()]
#Vemos las filas donde Revised Cost sea NaN. Esta linea lleva implicita la evaluación True

,Permit Number,Permit Type,Permit Creation Date,Description,Revised Cost
2,201605278609,3,05/27/2016,installation of separating wall,NaN
22,201708013544,3,08/01/2017,"soft story retrofit per sfebc chapter 4d, asce...",NaN
55,201710242043,7,10/24/2017,re-face (1) existing projecting sign. maher na,NaN
64,201711154040,3,11/15/2017,fire alarm system for appln #201006104250,NaN
82,201712196846,3,12/19/2017,to comply with nov 201719241 seismic retrofit....,NaN
...,...,...,...,...,...
198823,201802232134,6,02/23/2018,demolish 1-story gas station.** maher: n/a **,NaN
198827,201802232137,3,02/23/2018,infill (e) stair to add 70sf of conditioned sp...,NaN
198828,201802232137,3,02/23/2018,infill (e) stair to add 70sf of conditioned sp...,NaN
198836,201802232143,4,02/23/2018,erect non electric wall sign,NaN


Podemos intuir que la fila Description podría haber sido registrada, pero en 290 valores no lo fue, el dato es que no esta registrado o no hay descripción.

In [214]:
sin_nan = building_df_2[building_df_2['Revised Cost'].isna()== False]

In [245]:
building_df_2 = building_df_2.fillna(0) #Volvemos a declarar la variable, ahora con el valor de 0 en donde antes estaba NaN
building_df_2.head(5)

,Permit Number,Permit Type,Permit Creation Date,Description,Revised Cost
0,201505065519,4,05/06/2015,"ground fl facade: to erect illuminated, electr...",4000.0
1,201604195146,4,04/19/2016,remove (e) awning and associated signs.,500.0
2,201605278609,3,05/27/2016,installation of separating wall,0.0
3,201611072166,8,11/07/2016,repair dryrot & stucco at front of bldg.,2000.0
4,201611283529,6,11/28/2016,demolish retail/office/commercial 3-story buil...,100000.0
